<a href="https://colab.research.google.com/github/swjh9523/Perfect_Guide_Python/blob/main/05_Perfect_Guide_Cross_Val_Score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차검증을 보다 간편하게 - cross_val_score()

* KFold 클래스를 이용한 교차 검증 방법

    1. 폴드 세트 설정
    2. For 루프에서 반복적으로 학습/검증 데이터 추출 및 학습과 예측 수행
    3. 폴드 세트별로 예측 성능을 평균하여 최종 성능 평가

    ---> cross_val_score() 함수로 폴드 세트 추출, 학습/예측, 평가를 한번에 수행

    ```
    cross_val_score(estimator, X, y=None, scoring=None,cv=None,
    n_jobs = 1, verbose=0, fit_params=None, pre_dispatch='2*n_jobs')
    ```

# GridSearchCV - 교차 검증과 최적 하이퍼 파라미터 튜닝을 한번에

* 사이킷런은 GridSearchCV를 이용해 Classifier나 Regressor와 같은 알고리즘에 사용되는 하이퍼 파라미터를 순차적으로 입력하면서 편리하게 최적의 파라미터를 도출할 수 있는 방안을 제공합니다.

```
grid_parameters = {'max_depth' : [1, 2. 3],
'min_samples_split' : [2, 3]
}
```

ex) cv세트가 3이라면?
        파라미터 순차 적용횟수 6 * cv세트수 3 = 학습/검증 총 수행횟수 18

* cross_val_score()

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris
import numpy as np

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data
label = iris_data.target

# 성능 지표는 정확도(accuracy), 교차 검증세트 3개
scores = cross_val_score(dt_clf, data, label, scoring='accuracy', cv=3)
print('교차 검증별 정확도 : ', np.round(scores,4))
print('평균 검증 정확도 : ', np.round(np.mean(scores), 4))

교차 검증별 정확도 :  [0.98 0.94 0.98]
평균 검증 정확도 :  0.9667


* GridSearchCV

In [11]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target,
                                                    test_size = 0.2, random_state = 121)

dtree = DecisionTreeClassifier()

# parameter 들을 dictionary 형태로 설정

parameters = {'max_depth' : [1,2,3], 'min_samples_split' : [2,3]}

In [13]:
import pandas as pd

# param_grid의 하이퍼 파라미터들을 3개의 train, test set fold로 나누어서 테스트 수행 설정.
# refit=True가 default임. True이면 가장 좋은 파라미터 설정으로 재 학습 시킴.
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True, return_train_score=True)

# 붓꽃 Train 데이터로 param_grid의 하이퍼 파라미터들을 순차적으로 학습/평가
grid_dtree.fit(X_train, y_train)

# GridSearchCV 결과는 cv_results_라는 딕셔너리로 저장됨. 이를 DataFrame으로 변환
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score','rank_test_score','split0_test_score','split1_test_score','split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95
